# I want to actually make an LSTM
I will use the data in entero_unique_arg.csv, break each entry up into a list of 14 long characters that move over a space each time. The 15th character will be the one that the neural net is trying to predict. 


For each line of the input data I want to give it a 12 position long random "seed" along with a "start" character which I have decided is ":".

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

2021-10-14 11:02:52.236695: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-14 11:02:52.236741: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Tensor Flow Version: 2.4.0
Keras Version: 2.4.0

Python 3.8.12 | packaged by conda-forge | (default, Sep 29 2021, 19:52:28) 
[GCC 9.4.0]
Pandas 1.3.3
Scikit-Learn 1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


2021-10-14 11:02:56.546065: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-14 11:02:56.547957: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-14 11:02:56.547974: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-14 11:02:56.547996: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-UJ90D1J): /proc/driver/nvidia/version does not exist


In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)
pd.set_option('display.max_rows', 1000)
pd.set_option( 'display.max_colwidth', None )

df = pd.read_csv('entero_unique_arg.csv', skiprows=3, names=['sequences'])
FRAG_SIZE = 14

display(df.head())

,sequences
0,GCATCCGTAGTTCAGTTGGAtAGAGCACTCGGCTACGAACCGAGAGGtCGGAGGTTCAAATCCTTCCGGATGCA
1,GCGCTTGTAGCTCAGTTGGAtAGAGCGCTACCCTCCGAAGGTAGAGGcCTCAGGTTCGAATCCTGTCAAGCGCA
2,GCGCTCTTAGCTCAGATGGAtAGAGCAACGGCCTTCTAAGCCGTAGGtCATAGGTTCGAATCCTATAGAGCGCA
3,GCGCCCTTAGCTCAGTTGGAtAGAGCAACGACCTTCTAAGTCGTGGGcCGCAGGTTCGAATCCTGCAGGGCGCG
4,GTCCTCTTAGTTAAATGGAtATAACGAGCCCCTCCTAAGGGCTAAtTGCAGGTTCGATTCCTGCAGGGGACA


In [3]:
def add_prefix(sequence):
    prefix = np.random.rand(1,FRAG_SIZE-1).tolist()[0]
    prefix.append(':')
    prefix.extend(sequence)
    return prefix

def add_suffix(sequence):
    sequence.append(';')
    return sequence

def split_sequence(sequence):
    return [str(x) for x in sequence]

def make_X(sequence):
    steps = len(sequence)-FRAG_SIZE
    train = []
    for step in range(steps):
        train.append( sequence[step:FRAG_SIZE+step+1] )
    return train
    
def make_y(sequence):
    steps = len(sequence)-FRAG_SIZE+1
    predict = []
    for step in range(steps):
        predict.append( sequence[FRAG_SIZE+step-1] )
    return predict

def make_dummy(item):
    try:
        return uniqueChars[item]
    except KeyError:
        return item
            

In [161]:
example = [0.3745401188473625,
  0.9507143064099162,
  0.7319939418114051,
  0.5986584841970366,
  0.15601864044243652,
  0.15599452033620265,
  0.05808361216819946,
  0.8661761457749352,
  0.6011150117432088,
  0.7080725777960455,
  0.020584494295802447,
  0.9699098521619943,
  0.8324426408004217,
  ':',
  'G',
  'C',
  'A',
  'T',
  'C',
  'C',
  'G',
  'T',
  'A',
  'G',
  'T',
  'T',
  'C',
  'A',
  'G',
  'T',
  'T',
  'G',
  'G',
  'A',
  't',
  'A',
  'G',
  'A',
  'G',
  'C',
  'A',
  'C',
  'T',
  'C',
  'G',
  'G',
  'C',
  'T',
  'A',
  'C',
  'G',
  'A',
  'A',
  'C',
  'C',
  'G',
  'A',
  'G',
  'A',
  'G',
  'G',
  't',
  'C',
  'G',
  'G',
  'A',
  'G',
  'G',
  'T',
  'T',
  'C',
  'A',
  'A',
  'A',
  'T',
  'C',
  'C',
  'T',
  'T',
  'C',
  'C',
  'G',
  'G',
  'A',
  'T',
  'G',
  'C',
  'A',
  ';']

In [4]:
df['sequences'] = df['sequences'].map( split_sequence )
df['sequences'] = df['sequences'].map( add_prefix )
df['sequences'] = df['sequences'].map( add_suffix )
#display( df['sequences'].map( add_prefix ) )

In [5]:
X = []
y = []
for sequence in df['sequences']:
    X.extend(make_X(sequence))

In [6]:
columns = [f'X_{x}' for x in range(FRAG_SIZE)]
columns.append('y')

dataset = pd.DataFrame(X, columns=columns)
display(dataset.head())
dataset.shape

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.37454,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C,A
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C,A,T
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C,A,T,C


(16519, 15)

In [7]:
# Just looking how many tRNAs start with each letter
dataset[dataset['X_13']==':'].groupby('y')['y'].count()

y
A      3
C     11
G    202
T      6
Name: y, dtype: int64

In [8]:
# Lets get the basic stats that Jeff was getting
chars = df['sequences'].map(lambda calc: len(calc)).sum()
uniqueChars = {':': 0, 'A': 1, 'T': 2, 'G':3, 'C':4, 'a':5, 't':6, 'g':7, 'c':8, ';':9}
uniqueChars

{':': 0,
 'A': 1,
 'T': 2,
 'G': 3,
 'C': 4,
 'a': 5,
 't': 6,
 'g': 7,
 'c': 8,
 ';': 9}

In [9]:
dataset = dataset.applymap(make_dummy)

In [10]:
#dataset.to_csv('processed_LSTM_dataset_chars.csv', index=False)
dataset

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0,3
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3,4
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4,1
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1,2
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1.000000,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16514,4.000000,1.000000,1.000000,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3,4
16515,1.000000,1.000000,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3.000000,4,3
16516,1.000000,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3.000000,4.000000,3,4
16517,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3.000000,4.000000,3.000000,4,3


# I HAVE MADE MY DATASET
Now lets train the LSTM finally!!!

In [11]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

In [12]:
df = pd.read_csv('processed_LSTM_dataset.csv')
display(df.head())
df.shape

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0,3
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3,4
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4,1
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1,2
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1.000000,2,4


(16519, 15)

In [18]:
df = df.astype({'X_13':'float64'})
df.dtypes

X_0     float64
X_1     float64
X_2     float64
X_3     float64
X_4     float64
X_5     float64
X_6     float64
X_7     float64
X_8     float64
X_9     float64
X_10    float64
X_11    float64
X_12    float64
X_13    float64
y         int64
dtype: object

In [247]:
#Lets vectorize this (or not)
#We can also just leave it as our numbers

In [22]:
inputLength = len(  list(df.values.tolist())[1]  ) - 1
outputLength = len(uniqueChars)
rowLength = len(  list(df.values.tolist())  )
print('Input dim: {}\nOutput dim: {}\nTraining: {}'.format(inputLength, outputLength, rowLength))
#dataList = list(df.values.tolist())

Input dim: 14
Output dim: 10
Training: 16519


In [250]:
#print('Vectorization')
#x = np.zeros(( rowLength, inputLength, outputLength ), dtype=object)
#y = np.zeros(( rowLength, outputLength ), dtype=object)

#for i, sentence in enumerate(dataList):
#    sentence = sentence[:-1]
#    nextChar = sentence[-1]
#    for t, char in enumerate(sentence):
#        x[i,t,uniqueChars[char]] = 1
#    y[i,uniqueChars[nextChar]]

Vectorization
['0.3745401188473625', '0.9507143064099162', '0.7319939418114051', '0.5986584841970366', '0.15601864044243652', '0.15599452033620265', '0.05808361216819946', '0.8661761457749352', '0.6011150117432088', '0.7080725777960455', '0.020584494295802447', '0.9699098521619943', '0.8324426408004217', ':', 'G']


In [297]:
mask = df.columns.str.contains('X')
X = np.array( list(df.loc[:,mask].values.tolist()), dtype='float64' )
y = np.array( pd.get_dummies( list(df['y'].values.tolist()) ), dtype=np.bool )

In [298]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [306]:
print('Build model')
model = Sequential()
model.add(LSTM(128, input_shape=(inputLength, 1)))
model.add( Dense(outputLength-1, activation='softmax') )

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model


In [307]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 1161      
Total params: 67,721
Trainable params: 67,721
Non-trainable params: 0
_________________________________________________________________


In [261]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [331]:
# Left off at 16:31 of the lstm generate text video...
uniqueNums = {value:key for key,value in uniqueChars.items()}
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    #start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [1.0, 1.2, 1.5, 1.8, 2]:
        print('----- temperature:', temperature)

        generated = ''
        
        sentence = np.random.rand(1,FRAG_SIZE-1).tolist()[0]
        sentence.append(0)
        #print(sentence)
        #sentence = np.array(sentence, dtype='float64')
        #print(sentence)
        
        generated = ''
        print(f'----- Generating with seed: "{sentence}"')
        #sys.stdout.write(sentence)

        for i in range(80):
            x_pred = np.zeros((1,FRAG_SIZE,1))
            for t, char in enumerate(sentence):
                #try:
                 #   x_pred[0,t,0] = uniqueNums[char] ################################
                #except KeyError:
                x_pred[0,t,0] = char

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = uniqueNums[next_index]

            generated += next_char
            sentence = sentence[1:] + next_index

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [332]:
# Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# See https://github.com/tensorflow/tensorflow/issues/31308
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(X, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
130/130 [==============================] - 2s 16ms/step - loss: 0.1656
******************************************************
----- Generating text after Epoch: 0
----- temperature: 1.0
----- Generating with seed: "[0.16197692599669145, 0.46377558163211574, 0.32595704417649174, 0.14763643466486653, 0.6321110149600365, 0.9636727141315075, 0.6581094468542239, 0.3536986827773304, 0.7117969558099174, 0.14324815546973346, 0.5800163106934103, 0.469457446146198, 0.3599835714236381, 0]"
TT:T:GATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
----- temperature: 1.2
----- Generating with seed: "[0.03996761920917524, 0.353298349149453, 0.39823030414678107, 0.1546362003395415, 0.3441242849404804, 0.9344220150001339, 0.8585331039016134, 0.4459239610045047, 0.7098895698897182, 0.15184996190188538, 0.7525508593137176, 0.4757847699644099, 0.722568212030899, 0]"
T:TT:TGTTTTTTTTTGTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
----- temperature: 1.5
--

In [304]:
X.shape

(16519, 14, 1)

In [305]:
len(uniqueChars)

10

In [329]:
uniqueChars

{':': 0,
 'A': 1,
 'T': 2,
 'G': 3,
 'C': 4,
 'a': 5,
 't': 6,
 'g': 7,
 'c': 8,
 ';': 9}

In [330]:
uniqueNums

{0: ':',
 1: 'A',
 2: 'T',
 3: 'G',
 4: 'C',
 5: 'a',
 6: 't',
 7: 'g',
 8: 'c',
 9: ';'}